# For assigning chemical info and partial charges to PDB structures (saves to well-defined SDF files)

## Imports

In [5]:
# Generic Imports
from collections import defaultdict
import json

# Logging
import logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

logging.basicConfig(
    level=logging.WARNING,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)

# File I/O
from pathlib import Path
from tqdm.notebook import tqdm
from rich.progress import Progress, track # TODO : convert from tqdm

# Cheminformatics
from openff.toolkit.topology import Topology, Molecule

# Static Paths
MONO_DIR = Path('monomer_fragments')
PDB_DIR  = Path('pdb_files_new/trimers')

## Obtaining complete PDB-monomer pairs for iteration

In [ ]:
chem_paths = defaultdict(defaultdict)

for chem_mono in MONO_DIR.iterdir():
    chemistry = chem_mono.stem
    chem_pdb  = PDB_DIR / chemistry

    has_pdb  = set(path.stem for path in chem_pdb.glob( '*.pdb'))
    has_mono = set(path.stem for path in chem_mono.glob('*.json'))

    for valid_mol_name in (has_pdb & has_mono):
        pdb_path  = chem_pdb  / f'{valid_mol_name}.pdb'
        mono_path = chem_mono / f'{valid_mol_name}.json'

        chem_paths[chemistry][valid_mol_name] = (pdb_path, mono_path)

## Parameterizing and saving to SDF, recording mols where match fails

In [ ]:
from polymerist.openfftools import topology
from polymerist.openfftools.pcharge import MolCharger

from polymerist.monomers import MonomerGroup
from polymerist.residues.partition import partition

# catch annoying warnings
import warnings 
from openff.toolkit.utils.exceptions import IncorrectNumConformersWarning
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=IncorrectNumConformersWarning)

group_blacklist = [
    'poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine' # for some reason, these compounds are unable to be parameterized in a reasonable amount of time
]


topo_dir = Path('Topologies_new')
topo_dir.mkdir(exist_ok=True)

charge_method = 'Espaloma-AM1-BCC'
charger = MolCharger.subclass_registry[charge_method]()


unmatched_pdb_mols = defaultdict(defaultdict)
cancelled = defaultdict(list)
for chemistry, path_dict in chem_paths.items():
    chem_dir = topo_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    progress = tqdm(path_dict.items())
    for mol_name, (pdb_path, mono_path) in progress:
        progress.set_description_str(f'{chemistry} : {mol_name}')
        if any(group_name in mol_name for group_name in group_blacklist):
            progress.set_postfix_str('Cancelled')
            cancelled[chemistry].append(mol_name)
            continue

        try:
            progress.set_postfix_str('Loading files')
            monogrp = MonomerGroup.from_file(mono_path)
            offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers)
            progress.set_postfix_str('Partitioning')
            was_partitioned = partition(offtop)
            assert(was_partitioned)

            progress.set_postfix_str('Assigning Charges')
            offmol = topology.get_largest_offmol(offtop)
            offmol.name = mol_name
            cmol = charger.charge_molecule(offmol)

            progress.set_postfix_str('Saving SDF')
            offtop = cmol.to_topology()
            sdf_path = chem_dir / f'{mol_name}.sdf'
            topology.topology_to_sdf(sdf_path, offtop)
        except Exception as e:
            print(f'{mol_name} : {e}')
            unmatched_pdb_mols[str(e)][chemistry] = mol_name

In [8]:
cancelled

defaultdict(list,
            {'polyamide': ['poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine,benzene-1,3-dicarboxylic acid)',
              'poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine,4-[2-(4-carboxyphenyl)-1,1,1,3,3,3-hexafluoropropan-2-yl]benzoic acid)',
              'poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine,naphthalene-2,6-dicarboxylic acid)',
              'poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine,5-propan-2-ylbenzene-1,3-dicarboxylic acid)',
              'poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine,terephthalic acid)']})